# Basic Architecure for similarity model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os

# used below code for reading the dataset

# Specify the file path
csv_file_path = '/content/drive/MyDrive/image dataset/Data ID - Sheet1.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Create the target directory if it doesn't exist (in your Google Drive root directory)
target_directory = '/content/drive/MyDrive/final image dataset'
os.makedirs(target_directory, exist_ok=True)

# Loop through the DataFrame and download images
for index, row in df.iterrows():
    product_id = row['Product ID']
    image_url = row['image_link']
    response = requests.get(image_url)

    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        try:
          save_path = os.path.join(target_directory, f"{product_id}.jpg")
          image.save(save_path)

          print(f"Downloaded and saved {product_id}.jpg")
        except:
          pass
    else:
        print(f"Failed to download image for Product ID {product_id}")


Mounted at /content/drive
Downloaded and saved 1.jpg
Downloaded and saved 2.jpg
Downloaded and saved 3.jpg
Downloaded and saved 4.jpg
Downloaded and saved 5.jpg
Downloaded and saved 6.jpg
Downloaded and saved 7.jpg
Downloaded and saved 8.jpg
Downloaded and saved 9.jpg
Downloaded and saved 10.jpg
Downloaded and saved 11.jpg
Downloaded and saved 12.jpg
Downloaded and saved 13.jpg
Downloaded and saved 14.jpg
Downloaded and saved 15.jpg
Downloaded and saved 16.jpg
Downloaded and saved 17.jpg
Downloaded and saved 18.jpg
Downloaded and saved 19.jpg
Downloaded and saved 20.jpg
Downloaded and saved 21.jpg
Downloaded and saved 22.jpg
Downloaded and saved 23.jpg
Downloaded and saved 24.jpg
Downloaded and saved 25.jpg
Downloaded and saved 26.jpg
Downloaded and saved 27.jpg
Downloaded and saved 28.jpg
Downloaded and saved 29.jpg
Downloaded and saved 30.jpg
Downloaded and saved 31.jpg
Downloaded and saved 32.jpg
Downloaded and saved 33.jpg
Downloaded and saved 34.jpg
Downloaded and saved 35.jpg
Dow

In [2]:
print(len(df))

2074


In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.1 MB/s eta 0:00:00


In [4]:
import streamlit as st
import os
import numpy as np
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image


In [5]:
%%writefile sharedEmbed.pkl



Writing sharedEmbed.pkl


In [6]:

import pickle
model = ResNet50(weights='imagenet', include_top=False)
image_embeddings={}

def preprocess_image(image):
    img = image.resize((224, 224))
    img = np.array(img)  # Convert the image to a NumPy array
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img


def compute_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2)[0][0]


image_directory = '/content/drive/MyDrive/final image dataset'
image_files = os.listdir(image_directory)

count=0
for file in image_files:
    try:
      count+=1
      image_path = os.path.join(image_directory, file)
      img = Image.open(image_path)
      img = preprocess_image(img)
      embedding = model.predict(img)
      image_embeddings[file] = embedding
      # if count==1:
      #   break
    except:
      pass
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(image_embeddings, f)



1/1 [==============================] - 0s 178ms/step


In [7]:
import pickle
with open('embeddings.pkl', 'rb') as f:
    image_embeddings = pickle.load(f)
print(len(image_embeddings))

2072


In [8]:
%%writefile app.py
import streamlit as st
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import os
import io
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle


model = ResNet50(weights='imagenet', include_top=False)

with open('embeddings.pkl', 'rb') as f:
    image_embeddings = pickle.load(f)

def preprocess_image(image):
    img = image.resize((224, 224))
    img = np.array(img)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img

def compute_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2)[0][0]

def find_similar_images(uploaded_image, n_similar=5):
    uploaded_image = preprocess_image(uploaded_image)
    uploaded_embedding = model.predict(uploaded_image)
    uploaded_embedding = uploaded_embedding.reshape(1, -1)  # Reshape to match the shape of precomputed embeddings


    st.subheader(f"Uploaded Image Shape: {uploaded_embedding.shape}")
    similarities = {}

    for image_file, embedding in image_embeddings.items():

        embedding = embedding.reshape(1, -1)
        sim = compute_similarity(uploaded_embedding, embedding)
        similarities[image_file] = sim


        print(f"Computed Similarity for {image_file}: {sim:.2f}")

    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    top_similar_images = sorted_similarities[:n_similar]

    return top_similar_images






def main():
    st.title("Image Similarity Finder")

    st.sidebar.header("Instructions")
    st.sidebar.markdown("1. Upload an image.")
    st.sidebar.markdown("2. Click the 'Find Similar Images' button.")

    uploaded_image = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

    if uploaded_image is not None:

        uploaded_image = Image.open(uploaded_image)



        st.image(uploaded_image, caption="Uploaded Image", use_column_width=True)

        if st.button("Find Similar Images"):
            similar_images = find_similar_images(uploaded_image, n_similar=5)
            st.subheader(f"Found {len(similar_images)} Similar Images:")
            for i, (filename, sim) in enumerate(similar_images):
                st.subheader(f"Similar Image {i + 1} - Similarity Score: {sim:.2f}")
                similar_image = Image.open('/content/drive/MyDrive/final image dataset/' + filename)
                st.image(similar_image, caption=f"Similar Image {i + 1}", use_column_width=True)

if __name__ == "__main__":
    main()


Writing app.py


In [9]:
!streamlit run app.py & npx localtunnel --port 8501
# run this cell and open the url given in 'your url is ='. once you open this url, it will ask you for an Endpoint ID. For this purpose
# copy the External URL given and extract the part: 35.192.125.67 from http://35.192.125.67:8501. Hit enter and you will be able to seee
# the image.

[..................] - rollbackFailedOptional: verb npm-session c89a9e5999c41d4

npx: installed 22 in 5.591s
your url is: https://tricky-candies-check.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.5.254:8501

2024-02-04 13:59:44.456920: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 13:59:44.457063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 13:59:44.460703: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 13:59:45.940144: W tensorflow/compiler/tf2tensorrt/utils/